In [ ]:
import pandas as pd
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,

)
from tbdynamics.plotting import plot_model_vs_actual
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets
from multiprocessing import cpu_count

In [ ]:
pd.options.plotting.backend = "plotly"

## Define Model

### Params and calibration targets

In [ ]:
params = {
    'treatment_duration': 0.5, # 6 months
    'screening_start_asymp': 0.,
}

bcm = get_bcm(params)

### Running Optimization

In [ ]:
# self_recovery_constraint = lambda params: params["smear_negative_self_recovery"] - params["smear_positive_self_recovery"]
# # Create a suggested dictionary with constraints
# suggested = {
#     'constraint_self_recovery': self_recovery_constraint
# }
# # Get the number of CPU cores
# num_cores = cpu_count()
# # Set the number of workers for parallel optimization
# num_workers = num_cores
# opt_class = ng.optimizers.TwoPointsDE
# orunner = optimize_model(bcm, opt_class=opt_class,suggested=suggested, num_workers=num_cores)
# for i in range(8):
#     rec = orunner.minimize(1000)
# mle_params = rec.value[1]
# mle_params
   

In [ ]:
mle_params = {'start_population_size': 2025577.887771663,
 'contact_rate': 0.0065783907008427605,
 'rr_infection_latent': 0.23901945229141702,
 'rr_infection_recovered': 0.3832923417680706,
 'progression_multiplier': 1.0753407252985692,
 'seed_time': 1890.2161757589502,
 'seed_num': 98.18880812723295,
 'seed_duration': 4.6216749718551435,
 'smear_positive_death_rate': 0.34026379883361724,
 'smear_negative_death_rate': 0.03299684067196263,
 'smear_positive_self_recovery': 0.17706292549474267,
 'smear_negative_self_recovery': 0.11396767671244666,
 'screening_scaleup_shape': 0.09173535629367452,
 'screening_inflection_time': 1993.0373679804145,
 'screening_end_asymp': 0.641575480030553}

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs
targets = load_targets()

### Outputs

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['pop'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [ ]:

plot_model_vs_actual(
    derived_df_0, targets['incidence'], "incidence", "Incidence", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [ ]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notifs'], "notification", "Notification", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
derived_df_0['cdr'].plot()

In [ ]:
plot_model_vs_actual(derived_df_0, targets['prevalence_pulmonary'], 'prevalence_pulmonary', 'Prevalence Pulmonary', 'Modelled vs Estimation from 2017 prevalence survey')
